## Sentiment Analysis: classificatore di recensioni

Creare un dataset di recensioni neutre con chat-gpt? (quanto costa?)

Un dataset di 25.000 recensioni di film più o meno recenti (per il <u>training</u>):<br>
* una linea *logica* per recensione (ritorno a capo voluto).<br>
* le prime 12500 sono state classificate come positive, le seconde 12.500 come negative.

Un secondo dataset con altre 25.000 recensioni di <u>test</u>, strutturate analogamente a quelle di training.  

I due dataset sono scaricabili dal sito della [Fordham University](https://www.fordham.edu/about/) ([qui](https://storm.cis.fordham.edu/~yli/data/movie_data/)).

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import re

C:\Users\Utente\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Caricamento dei dati

In [3]:
reviews_train = []                                                        # un'array vuota

for line in open ('movie_review_full_train.txt','r',encoding='cp437'):    # il loop di caricamento dell'array
                 reviews_train.append(line.strip())

In [4]:
reviews_train[1:10] # contiene anche molte informazioni non utili all'analisi: spazi, lettere maiuscole, congiunzioni, 
              # disgiunzioni, punteggiatura, tag HTML, ecc

['Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they\'ll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it\'s like to be homeless? That is Goddard Bolt\'s lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days witho

In [6]:
reviews_test = []
for line in open ('movie_review_full_test.txt','r',encoding='cp437'):
                 reviews_test.append(line.strip())

In [7]:
reviews_test[1:10]  

['Actor turned director Bill Paxton follows up his promising debut, the Gothic-horror "Frailty", with this family friendly sports drama about the 1913 U.S. Open where a young American caddy rises from his humble background to play against his Bristish idol in what was dubbed as "The Greatest Game Ever Played." I\'m no fan of golf, and these scrappy underdog sports flicks are a dime a dozen (most recently done to grand effect with "Miracle" and "Cinderella Man"), but some how this film was enthralling all the same.<br /><br />The film starts with some creative opening credits (imagine a Disneyfied version of the animated opening credits of HBO\'s "Carnivale" and "Rome"), but lumbers along slowly for its first by-the-numbers hour. Once the action moves to the U.S. Open things pick up very well. Paxton does a nice job and shows a knack for effective directorial flourishes (I loved the rain-soaked montage of the action on day two of the open) that propel the plot further or add some unexpe

In caso di errore di lettura dei file del tipo *UnicodeDecodeError* vedi [questo post stackoverflow](https://stackoverflow.com/questions/9233027/unicodedecodeerror-charmap-codec-cant-decode-byte-x-in-position-y-character).

### Pulizia dei dati

In [8]:
# le regole di pulizia (le uniche da aggiornare con un altro dataset)
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

In [9]:
# una funzione che:
# - applica le regole a tutti gli elementi delle due array di recensioni prima caricate
# - converte in minuscolo ogni stringa
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    return reviews

In [10]:
reviews_train_clean = preprocess_reviews(reviews_train)

In [11]:
reviews_train_clean[1:10]

['homelessness or houselessness as george carlin stated has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or vote for the matter most people think of the homeless as just a lost cause while worrying about things such as racism the war on iraq pressuring kids to succeed technology the elections inflation or worrying if theyll be next to end up on the streets but what if you were given a bet to live on the streets for a month without the luxuries you once had from a home the entertainment sets a bathroom pictures on the wall a computer and everything you once treasure to see what its like to be homeless that is goddard bolts lesson mel brooks who directs who stars as bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival jeffery tambor to see if he can live in the streets for thirty days without the luxuries if bolt succeeds he can do what he 

In [11]:
reviews_test_clean = preprocess_reviews(reviews_test)

### Vettorizzazione dei testi

[CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) è una funzione di *scikit-learn* che permette di convertire testo in un vettore di frequenze dei termini (*token*).<br><br>
Crea cioè una [**bag of words**](https://en.wikipedia.org/wiki/Bag-of-words_model), anche detta [**matrice documenti-termini**](https://en.wikipedia.org/wiki/Document-term_matrix).<br><br>
Preliminarmente il vettorizzatore deve eseguire una [**analisi lessicale**](https://it.wikipedia.org/wiki/Analizzatore_lessicale), per isolare i singoli termini (da non confondere con l'analisi sintattica, cioè la [*parsificazione*](https://it.wikipedia.org/wiki/Parsing)).

In [12]:
cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)      # la matrice di training (una serie di vettori, uno per recensione)
X_test = cv.transform(reviews_test_clean)  # la matrice di test

In [13]:
X.shape  # la matrice recensioni-termini

(25000, 91099)

In [14]:
X[1]  

<1x91099 sparse matrix of type '<class 'numpy.int64'>'
	with 230 stored elements in Compressed Sparse Row format>

### Un modello di classificazione regolarizzata 
Useremo la Regressione Logistica (in forma [regolarizzata](https://it.wikipedia.org/wiki/Regolarizzazione_(matematica))).<br>
La regolarizzazione scoraggia con una penalità i coefficienti grandi (e troppi) del modello.<br>
In altre parole, la regolarizzazione è una tecnica per costruire modelli semplici e non inutilmente complessi.

In [15]:
# creazione del vettore delle y  --> 1: recensione positiva, 0: recensione negativa:
target = [1 if i < 12500 else 0 for i in range(25000)] 
target # 1: recensione positiva (le prime 12.500)
       # 0: recensione negativa (le altre 12.500)


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [16]:
# suddivisione della matrice di training in una matrice di training ed in una matrice di "validazione" (per il tuning 
# dell'iper-parametro C - vedi più avanti)
X_train, X_val, y_train, y_val = train_test_split(X,target,train_size=0.75,random_state=1) 

In [17]:
y_train

[0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,


In [18]:
pd.DataFrame({'Frequency table': y_train}).value_counts()  # le frequenze di 0 ed 1;
                                                           # [la funzione 'value_counts()' di pandas calcola la frequency 
                                                           # table; 'y_train' è una 'lista', e quindi dev'essere prima 
                                                           # convertita in un dataframe]

Frequency table
0                  9400
1                  9350
dtype: int64

In [19]:
25000*0.25

6250.0

In [20]:
X_train

<18750x91099 sparse matrix of type '<class 'numpy.int64'>'
	with 2593805 stored elements in Compressed Sparse Row format>

In [21]:
# model assessment sulla matrice di validazione:
for c in [0.01, 0.05, 0.25, 0.5, 1]:                       # valori dell'iper-parametro C
    lr = LogisticRegression(C=c,max_iter=1000)
              # C : float, default=1.0
              # Inverse of regularization strength; must be a positive float.
              # Like in support vector machines, smaller values specify stronger regularization.
    lr.fit(X_train,y_train)                                # fit sulla matrice di training (ridotta)
    print ("Accuracy for C=%s: %s"
           % (c,accuracy_score(y_val,lr.predict(X_val))))  # score sulla matrice di validazione (per la comparazione)

Accuracy for C=0.01: 0.8736
Accuracy for C=0.05: 0.88304
Accuracy for C=0.25: 0.88176
Accuracy for C=0.5: 0.88208
Accuracy for C=1: 0.87856


In [22]:
# l'accuratezza predittiva (sulla matrice di test) del modello migliore
final_model = LogisticRegression(C=0.05,max_iter=1000)
final_model.fit(X,target)                                     # fit sulla matrice di training COMPLETA
print ("Final Accuracy: %s"
       % accuracy_score(target,final_model.predict(X_test)))  # score sulla matrice di test

Final Accuracy: 0.8814


In [27]:
# positive values: good review; 0-negative values: bad review
print(final_model.predict(cv.transform([input()]))[0])
                   # "what a boring film" --> 0
                   # "I was really annoyed by this film" --> 0
                   # "great" --> 1
                   # "interesting film" --> 1
                   # "on the whole this movie is not entirely unentertaining but it is majorly disappointing" --> 0
                   # "a good but too long film" --> 1 (perchè manca la terza classe neutra)

a good but too long film
1


Questo classificatore funziona per **qualsiasi tipo di testi** (recensioni di film, recensioni di prodotti, post sul sito aziendale, ecc), in **qualsiasi lingua**, in **qualsiasi dominio applicativo**; occorre solo personalizzare la <u>lista di caratteri speciali</u> che si devono eliminare, che dipende ovviamente dal contesto.<br><br>
Al contrario, con un [sistema esperto anni '90](https://it.wikipedia.org/wiki/Sistema_esperto), il dizionario dei termini e della loro caratterizzazione (ad es. positivo, negativo, neutro) deve essere preparato **in anticipo** (consultando gli esperti del dominio applicativo) e **dipende dalla lingua**.

### Estrazione dei termini che sono stati classificati positivi o negativi

In [23]:
feature_to_coef = {
    word: coef for word, coef in zip(
    cv.get_feature_names(),final_model.coef_[0]
    )
}

I termini con punteggi vicini ad 1 sono considerati *positivi*, quelli con punteggi negativi sono considerati *negativi*.<br>
Alle nuove recensioni possiamo così attribuire un punteggio che ci permette di valutare se la recensione nel suo complesso esprime un *sentiment* positivo o negativo. 

In [24]:
for best_positive in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1],
    reverse=True)[:5]:
    print (best_positive)

('excellent', 0.9272440569670202)
('perfect', 0.7949921784196641)
('great', 0.6750559502017355)
('amazing', 0.61607994028369)
('superb', 0.6066433909193281)


In [25]:
for best_negative in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1])[:5]:
    print (best_negative)

('worst', -1.3676235208380287)
('waste', -1.1685453920483562)
('awful', -1.0265825658995213)
('poorly', -0.8732649370748898)
('boring', -0.8592868584663386)
